머신러닝 모델 평가의 심화 과정으로, 단순한 수치를 넘어 모델의 '전반적인 능력치'를 평가하는 'ROC 곡선과 AUC 점수'를 다룬다.

이 시간의 핵심은 "임계값(Threshold)이 변할 때마다 모델의 성능이 어떻게 출렁이는가?"를 시각화하고, 이를 통해 "그래서 최적의 진단 기준점은 어디인가?"를 결정하는 방법을 배우는 것이다.

---

# 📘 [2일차 5교시] 5.1. 분류 모델 평가 지표 2: ROC 곡선과 AUC - 모델의 종합 능력 평가

## 1. 정밀도와 재현율의 한계: "기준이 바뀌면 점수도 바뀐다"

### 1) 고정된 기준(Threshold)의 불안함
앞선 4교시에서 우리는 정밀도(Precision)와 재현율(Recall)을 배웠다. 이 두 지표는 모델의 성능을 아주 잘 보여주지만, 한 가지 치명적인 약점이 있다. 바로 "임계값(Threshold)을 어디에 두느냐에 따라 점수가 널뛰기를 한다"는 점이다.

우리가 개발한 암 진단 AI 모델이 있다고 가정해 보자. 이 모델은 환자의 데이터를 보고 '암일 확률(0~1)'을 내뱉는다.
*   상황 A (엄격한 기준): "확률이 0.9 이상일 때만 암이라고 하자."
    *   아주 확실한 환자만 잡는다. 오진(FP)은 없겠지만, 놓치는 환자(FN)가 많다.
    *   결과: 정밀도는 높지만, 재현율은 낮다.
*   상황 B (너그러운 기준): "확률이 0.1 이상이면 일단 암이라고 하자."
    *   조금이라도 의심되면 다 잡는다. 놓치는 환자(FN)는 없겠지만, 멀쩡한 사람을 잡는 오진(FP)이 폭증한다.
    *   결과: 재현율은 높지만, 정밀도는 낮다.

이때 병원장님이 묻는다. "그래서, 이 AI 모델의 점수가 몇 점인가? 좋은 모델인가, 나쁜 모델인가?"
상황 A를 보면 나쁜 모델 같고, 상황 B를 보면 또 다르게 보인다. 기준선 하나에 따라 평가가 달라진다면, 모델 자체의 '순수한 잠재력(Potential)'을 평가하기가 어렵다.

### 2) 전체를 아우르는 지표의 필요성
우리는 특정 기준(0.5 등)에서의 점수가 아니라, "이 모델이 가질 수 있는 모든 기준에서의 성능을 종합적으로 보고 싶다"는 욕구가 생긴다.
*   "기준을 아주 엄격하게 했을 때부터 아주 느슨하게 했을 때까지, 이 모델은 전반적으로 밥값을 하는가?"
*   이 질문에 답하기 위해 만들어진 도구가 바로 ROC 곡선이다.

---

## 2. ROC 곡선: 레이더병의 딜레마에서 탄생한 곡선

### 1) 역사적 배경: 적기인가, 새떼인가?
ROC(Receiver Operating Characteristic)라는 이름은 2차 세계대전 당시 레이더 통신병들의 능력을 평가하기 위해 만들어졌다.
레이더 화면에는 아군을 위협하는 적기(Signal)와, 구름이나 새떼 같은 노이즈(Noise)가 뒤섞여서 희미한 점으로 나타난다. 레이더병은 이 점을 보고 판단을 내려야 한다.

*   민감한 병사: 점이 조금만 반짝여도 "적기 출현! 출격하라!"고 외친다.
    *   장점: 적기가 침투할 틈을 주지 않는다. (재현율 100%)
    *   단점: 새떼를 보고 출격하는 일이 잦아 연료 낭비가 심하다. (위양성 증가)
*   신중한 병사: 점이 아주 확실하게 크고 진할 때만 보고한다.
    *   장점: 허탕 치는 일이 없다. (위양성 0%)
    *   단점: 흐릿하게 날아오는 적기를 놓쳐서 기지가 폭격당할 수 있다. (재현율 감소)

헬스케어의 AI 모델도 이 레이더병과 똑같은 딜레마에 빠져 있다. MRI 사진의 흐릿한 음영을 보고 '암(Signal)'인지 '정상 조직(Noise)'인지 판별해야 한다. ROC 곡선은 이 모델이 "가장 민감할 때부터 가장 신중할 때까지" 보여주는 모든 성적표를 하나의 그래프로 그린 것이다.

### 2) ROC 곡선의 두 축: TPR과 FPR
ROC 곡선은 $x$축과 $y$축에 다음과 같은 지표를 놓고 그린다.

#### $y$축: TPR (True Positive Rate) = 재현율(Recall) = 민감도(Sensitivity)
*   의미: "실제 환자 중에 AI가 제대로 찾아낸 비율"이다.
*   목표: 높을수록 좋다. (위로 갈수록 좋음)
*   해석: "우리 AI는 환자를 놓치지 않고 싹 다 잡아들입니다."

#### $x$축: FPR (False Positive Rate) = 위양성률 = (1 - 특이도)
*   의미: "실제 정상인 중에 AI가 환자라고 오진한 비율"이다.
*   목표: 낮을수록 좋다. (왼쪽으로 갈수록 좋음)
*   해석: "우리 AI는 멀쩡한 사람을 붙잡고 환자라고 우기지 않습니다."

결론: 가장 좋은 모델은 그래프의 왼쪽 위 구석(좌상단)에 점이 찍히는 모델이다. 즉, "오진(FPR)은 하나도 없으면서 환자(TPR)는 다 찾아내는" 신의 경지이다.

---

## 3. 곡선 그리기: 임계값(Threshold) 슬라이더 밀기

ROC 곡선은 하나의 점이 아니라 '선'이다. 이 선은 임계값을 1.0에서 0.0으로 서서히 내리면서 그려진다. 이 과정을 상상해 보자.

### 1) 단계 1: 아주 엄격할 때 (Threshold = 1.0)
*   AI의 기준: "확률이 100%가 아니면 환자가 아니다."
*   행동: 아무도 환자라고 지목하지 않는다.
*   결과: 환자를 한 명도 못 찾음(TPR=0), 오진도 없음(FPR=0).
*   좌표: $(0, 0)$에서 시작한다.

### 2) 단계 2: 기준을 조금 낮춤 (Threshold = 0.8)
*   AI의 기준: "80% 이상 확실하면 환자라고 하자."
*   행동: 아주 확실한 암 환자들을 찾아내기 시작한다.
*   결과: 환자 검거율(TPR)이 쑥 올라간다. 워낙 확실한 것만 잡았으므로 오진(FPR)은 거의 늘지 않는다.
*   좌표: 그래프가 위쪽(y축 방향)으로 급격히 상승한다.

### 3) 단계 3: 기준을 더 낮춤 (Threshold = 0.5)
*   AI의 기준: "반반이면 환자로 보자."
*   행동: 애매한 환자들도 잡아들인다. 동시에 정상인 중에서도 암과 비슷하게 생긴 사람들을 오진하기 시작한다.
*   좌표: 그래프가 오른쪽(x축 방향)으로 조금씩 휘어지기 시작한다.

### 4) 단계 4: 아주 느슨할 때 (Threshold = 0.0)
*   AI의 기준: "그냥 다 환자라고 해!"
*   행동: 세상 모든 사람을 환자라고 진단한다.
*   결과: 실제 환자를 100% 찾았다(TPR=1). 하지만 모든 정상인을 오진했다(FPR=1).
*   좌표: $(1, 1)$에서 끝난다.

이 점들을 이었을 때, 그래프가 위쪽으로 불룩하게 솟아오를수록 좋은 모델이다. 반대로 대각선(45도 선)을 따라간다면, 그것은 동전 던지기(랜덤)와 다를 바 없는 멍청한 모델이라는 뜻이다.

---

## 4. AUC (Area Under the Curve): 모델의 등급 매기기

곡선의 모양만 보고 "이게 저것보다 낫다"고 말하기는 애매하다. 그래서 수학적으로 명확하게 점수를 매기기 위해 곡선 아래의 면적을 계산한다. 이것이 AUC (Area Under the Curve)이다.

### 1) AUC 점수의 의미
AUC는 0에서 1 사이의 값을 가진다.
*   AUC = 0.5: 대각선. 모델이 아무런 학습도 하지 못했다. 찍기 수준이다.
*   AUC = 0.7 ~ 0.8: 어느 정도 쓸만한 모델 (Fair).
*   AUC = 0.8 ~ 0.9: 훌륭한 모델 (Good). 헬스케어 연구에서 목표로 하는 구간이다.
*   AUC = 0.9 ~ 1.0: 완벽에 가까운 모델 (Excellent).
    *   *주의:* 만약 학습하자마자 AUC가 1.0이 나왔다면, "내가 노벨상감이구나"라고 생각하기보다 "데이터에 정답이 섞여 들어간 게 아닐까(Data Leakage)?"라고 의심해야 한다. 현실 세계에서 완벽한 1.0은 거의 불가능하다.

### 2) 확률적 해석 (직관적 이해)
AUC가 0.8이라는 것은 무슨 뜻일까?
이것은 "랜덤하게 암 환자 1명과 정상인 1명을 뽑았을 때, AI가 암 환자의 위험도(확률)를 정상인보다 더 높게 계산했을 확률이 80%이다"라는 뜻이다.
즉, AUC는 모델이 "누가 더 위험한지 줄 세우기(Ranking)를 얼마나 잘하는가"를 나타내는 척도이다.

### 3) 헬스케어에서 AUC가 사랑받는 이유
*   데이터 불균형에 강하다: 정확도(Accuracy)는 환자가 적으면 왜곡되지만, AUC는 환자가 적든 많든 모델의 변별력을 공정하게 평가한다.
*   종합적 판단: 특정 병원은 민감도를 중시하고, 특정 병원은 특이도를 중시할 수 있다. AUC는 이 모든 상황을 아울러서 "이 모델은 기본 체급(능력치)이 좋다"는 것을 증명해 준다. 그래서 FDA 승인이나 의학 논문에서는 모델 성능의 증거로 반드시 AUC 값을 요구한다.

---

## 5. 최적의 임계값 (Optimal Threshold) 찾기

AUC가 높다는 것은 모델이 '잠재력'이 좋다는 뜻이다. 하지만 실제 병원 현장에 이 AI를 설치하려면, 결국 "그래서 몇 점 이상을 암으로 볼 건데?"라는 하나의 기준값을 정해줘야 한다.

### 1) 좌상단에 가장 가까운 점 (Youden's J Statistic)
ROC 곡선 위에서 $(0, 1)$ 좌표, 즉 왼쪽 위 꼭짓점에 가장 가까운 점을 찾는다.
이 지점이 "환자는 최대한 많이 잡으면서(TPR 최대화), 오진은 최소로 줄이는(FPR 최소화)" 황금 밸런스를 가진 지점이다.

### 2) 비용 기반 결정 (Cost-based Decision)
때로는 수학적 최적점이 정답이 아닐 수도 있다.
*   전염병 판별: 오진(격리 해제 후 재검사) 비용보다 놓침(전파) 비용이 훨씬 크다. 이때는 최적점보다 임계값을 더 낮춰서라도 민감도를 확보해야 한다.
*   이처럼 ROC 곡선을 그려놓고, 병원의 정책과 비용을 고려하여 최종 임계값을 선택하는 것이 데이터 과학자의 역할이다.

---

## 6. 요약: 5교시의 핵심 메시지

1.  ROC 곡선: 임계값을 1에서 0으로 변화시키며, 모델의 검거율(TPR)과 오진율(FPR)의 변화를 그린 그래프이다.
2.  모양: 그래프가 왼쪽 위로 바짝 붙을수록 좋은 모델이다. (오진 없이 잘 잡는다는 뜻)
3.  AUC (면적): ROC 곡선 아래의 면적으로, 모델의 성능을 0.5~1.0 사이의 점수로 나타낸 것이다. 헬스케어 모델 평가의 표준 지표이다.
4.  강건함(Robustness): AUC는 데이터 불균형이나 임계값 설정에 상관없이 모델의 순수한 변별력을 평가한다.
5.  활용: 모델 간의 우열을 가릴 때(모델 A vs 모델 B) 가장 먼저 확인해야 할 지표이며, 최종적으로는 이 곡선 위에서 상황에 맞는 최적의 임계값을 찾아내야 한다.

이 이론적 배경을 바탕으로, 다음 시간에는 코드를 통해 직접 ROC 곡선을 그려보고 내 모델의 AUC 점수를 계산해 보겠다.

## 5.2. \[예제 1\] F1-Score: 불균형 데이터의 구세주

ROC로 넘어가기 전, 4교시의 연장선에서 정밀도와 재현율의 조화 평균인
F1-Score의 위력을 확인한다.

상황: 암 환자가 10%뿐인 데이터에서 정확도와 F1-Score 비교.

``` python
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# 1. 불균형 데이터 생성 (100명 중 10명만 환자)
# 실제값: 0(정상) 90명, 1(환자) 10명
y_true = np.array([0]*90 + [1]*10)

# 2. 모델 A: 무조건 '정상(0)'이라고 찍는 바보 모델
y_pred_dummy = np.array([0]*100)

# 3. 모델 B: 환자 10명 중 6명을 찾고, 정상인 5명을 오진한 모델 (현실적인 모델)
# 예측: 0(정상) 85명, 1(환자) 5명(오진), 0(정상) 4명(놓침), 1(환자) 6명(찾음)
# 순서는 섞여있지만 개수로 비교
y_pred_smart = np.array([0]*85 + [1]*5 + [0]*4 + [1]*6)

# 4. 성능 비교
print("=== 모델 A (무조건 정상) ===")
print(f"정확도: {accuracy_score(y_true, y_pred_dummy):.2f}")
print(f"F1점수: {f1_score(y_true, y_pred_dummy):.2f}")

print("\n=== 모델 B (노력하는 모델) ===")
print(f"정확도: {accuracy_score(y_true, y_pred_smart):.2f}")
print(f"F1점수: {f1_score(y_true, y_pred_smart):.2f}")

# 설명:
# 모델 A는 정확도가 0.90이나 되지만, F1점수는 0.0이다. (환자를 하나도 못 찾음 -> 쓰레기)
# 모델 B는 정확도가 0.91로 A와 비슷해 보이지만, F1점수는 약 0.57이다.
# 헬스케어에서는 F1점수가 0이 아닌 모델 B를 선택해야 한다.
```


> 📖 문제 풀이 가이드
>
> 4교시에 배웠던 F1-Score를 다시 한 번 복습하며, 불균형 데이터에서 왜 중요한지 확인합니다.
> - 모델 A (바보 모델): 100명 중 90명이 정상인 데이터에서 무조건 "정상"만 외칩니다. 정확도는 90%지만 F1점수는 0점입니다.
> - 모델 B (노력 모델): 환자 10명 중 6명을 찾았지만, 오진도 좀 있습니다. 정확도는 비슷하게 91%지만, F1점수는 0.57점입니다.
> - 결론: 정확도가 비슷해도 F1-Score가 높은 모델 B가 훨씬 가치 있는 모델입니다.



In [ ]:
# do write code


## 5.3. \[예제 2\] ROC 곡선 그리기 (Scikit-learn)

Scikit-learn의 `roc_curve` 함수를 이용해 TPR과 FPR의 변화를 시각화한다.

상황: 심장병 예측 모델의 성능을 시각적으로 검증.

``` python
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score

# 1. 가상 분류 데이터 생성 (불균형 설정)
X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.9, 0.1], random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 2. 모델 학습 (로지스틱 회귀)
model = LogisticRegression()
model.fit(X_train, y_train)

# 3. 확률 예측 (중요: 0/1 클래스가 아니라 '확률'이 필요함)
# predict_proba의 두 번째 컬럼(1일 확률)을 가져온다.
y_prob = model.predict_proba(X_test)[:, 1]

# 4. ROC 곡선 데이터 계산
# thresholds: 임계값을 1.0에서 0.0으로 내리면서 계산된 FPR, TPR 값들
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# 5. 시각화
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='red', label='Logistic Regression')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--', label='Random Guess') # 기준선
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.title('ROC Curve')
plt.legend()
plt.show()

# 설명:
# 빨간 곡선이 파란 점선(랜덤)보다 위쪽으로 많이 휘어질수록 좋은 모델이다.
# 왼쪽 위 모서리에 가까울수록 완벽에 가깝다.
```


> 📖 문제 풀이 가이드
>
> 이제 모델의 성적표를 표(Table)가 아닌 그림(Curve)으로 그려봅니다. 이것이 ROC 곡선입니다.
> - X축 (FPR): 오진율 (낮을수록 좋음, 왼쪽)
> - Y축 (TPR): 재현율 (높을수록 좋음, 위쪽)
> - 형태: 왼쪽 아래(0,0)에서 시작해 오른쪽 위(1,1)로 끝나는 곡선입니다.
> - 해석: 곡선이 왼쪽 위 모서리(좌상단)에 바짝 붙을수록 훌륭한 모델입니다.



In [ ]:
# do write code


## 5.4. \[예제 3\] AUC Score 비교: 누가 더 훌륭한가?

두 개의 서로 다른 모델을 만들고, AUC 점수로 우열을 가린다.

상황: 기존 모델(A)과 신규 모델(B) 중 어떤 것을 도입할지 결정.

``` python
from sklearn.ensemble import RandomForestClassifier

# 1. 모델 A: 로지스틱 회귀 (위에서 학습함)
auc_logistic = roc_auc_score(y_test, y_prob)

# 2. 모델 B: 랜덤 포레스트 (일반적으로 성능이 더 좋음)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]
auc_rf = roc_auc_score(y_test, y_prob_rf)

# 3. 결과 출력
print(f"로지스틱 회귀 AUC: {auc_logistic:.4f}")
print(f"랜덤 포레스트 AUC: {auc_rf:.4f}")

# 4. 시각화 비교
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='red', label=f'Logistic (AUC={auc_logistic:.2f})')
plt.plot(fpr_rf, tpr_rf, color='green', label=f'Random Forest (AUC={auc_rf:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.title('ROC Curve Comparison')
plt.legend()
plt.show()

# 설명:
# 초록색 선(Random Forest)이 빨간 선보다 더 바깥쪽(왼쪽 위)에 있으므로,
# 랜덤 포레스트가 더 우수한 모델임을 한눈에 알 수 있다.
```


> 📖 문제 풀이 가이드
>
> 눈대중으로 그림만 보고 "이게 더 낫네" 하기에는 애매합니다. 그래서 면적을 계산해 점수로 만듭니다. 이것이 AUC(Area Under the Curve)입니다.
> - AUC = 0.5: 동전 던지기 수준 (공부 하나도 안 한 모델)
> - AUC = 1.0: 신의 경지 (모든 환자를 완벽하게 분류)
> - 비교: 로지스틱 회귀와 랜덤 포레스트 중 누가 더 AUC 점수가 높은지 확인해보세요.



In [ ]:
# do write code


## 5.5. \[예제 4\] 최적의 임계값(Optimal Threshold) 찾기

ROC 곡선을 그리는 것만으로는 부족하다. 그래서 실제 병원에서는
“임계값을 0.5가 아니라 0.3으로 설정해야 한다”는 구체적인 가이드가
필요하다. 가장 많이 쓰이는 Youden’s J statistic 방법을 사용한다. \*
J = Sensitivity + Specificity - 1 = TPR - FPR \* 이 J값이 최대가
되는 지점이 트레이드오프의 최적점이다.

``` python
import numpy as np

# 1. Youden's Index 계산
# tpr - fpr 이 가장 큰 지점이 최적이다.
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print("=== 최적의 임계값 찾기 ===")
print(f"Best Threshold: {optimal_threshold:.4f}")
print(f"이때의 TPR(재현율): {tpr[optimal_idx]:.4f}")
print(f"이때의 FPR(위양성률): {fpr[optimal_idx]:.4f}")

# 2. 시각화에 최적점 표시
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label='ROC Curve')
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='o', color='black', s=100, label='Best Cut-off')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Optimal Threshold on ROC')
plt.legend()
plt.show()

# 설명:
# 까만 점이 찍힌 곳이 "환자도 잘 잡고 오진도 적은" 밸런스가 가장 좋은 지점이다.
# 의사 결정 시 이 확률값(예: 0.32)을 기준으로 진단을 내리면 된다.
```


> 📖 문제 풀이 가이드
>
> ROC 곡선은 "우리 모델 성능이 이 정도야"를 보여줄 뿐, "그래서 임계값을 몇으로 해야 해?"는 알려주지 않습니다.
> - Youden's Index: (재현율 + 특이도 - 1) 값이 최대가 되는 지점. 즉, "잘 맞추면서(TPR) 오진(FPR)은 적은" 최적의 밸런스 포인트를 찾습니다.
> - 결과: 만약 `Best Threshold`가 0.3으로 나온다면, 앞으로는 확률이 30%만 넘어도 환자로 분류하면 됩니다.



In [ ]:
# do write code


## 5.6. \[예제 5\] 정밀도-재현율 곡선 (PR Curve)

데이터가 극도로 불균형할 때(예: 암 환자 0.1%)는 ROC 곡선조차도
과대평가될 수 있다. 이때는 PR Curve를 확인해야 한다.

상황: 정상인(0)이 압도적으로 많은 데이터에서, 환자(1) 클래스에만
집중하여 평가.

``` python
from sklearn.metrics import precision_recall_curve, average_precision_score

# 1. PR Curve 계산
precision, recall, _ = precision_recall_curve(y_test, y_prob_rf)
# AP Score (PR Curve 아래 면적)
ap_score = average_precision_score(y_test, y_prob_rf)

# 2. 시각화
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='purple', label=f'AP Score={ap_score:.2f}')
plt.xlabel('Recall (Sensitivity)')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
plt.show()

# 설명:
# ROC는 (1, 1)로 가지만, PR Curve는 (1, 1)인 오른쪽 위를 향해 갈수록 좋다.
# 재현율(Recall)을 높일 때 정밀도(Precision)가 얼마나 뚝뚝 떨어지는지를 적나라하게 보여준다.
# 희귀 질환 연구에서는 ROC보다 이 그래프가 더 엄격하고 유용하다.
```


> 📖 문제 풀이 가이드
>
> 데이터가 극단적으로 불균형할 때(예: 암 환자가 1000명 중 1명), ROC 곡선조차도 우리를 속일 수 있습니다. 이때 마지막 보루가 PR Curve(정밀도-재현율 곡선)입니다.
> - 특징: ROC와 달리 (1, 1)이 목표 지점이 아니라, 오른쪽 위(1, 1)가 목표 지점입니다.
> - 해석: 재현율(Recall)을 높이려고 할 때 정밀도(Precision)가 얼마나 급격하게 무너지는지를 가감 없이 보여줍니다.



In [ ]:
# do write code
